# Carga de datos para Endpoints de la API

Vamos a crear un dataframe para cada endpoint de la API que estamos construyendo. Estos dataframes se almacenarán en un archivo **.parquet** que será utilizado en las llamadas a la API, con el objetivo de mejorar el rendimiento.

Importamos las librerias necesarias

In [52]:
import pandas as pd
from datetime import datetime 

### 1. Endpoint PlayTimeGenre

In [53]:
df_users_items = pd.read_csv('../Data/Data-Extraida/users_items.csv')
df_users_items.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10.0,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20.0,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30.0,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40.0,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50.0,Half-Life: Opposing Force,0.0,0.0


Vamos a agrupar los datos por nombre del juego y sumar las horas totales jugadas para cada juego.

In [54]:
df_horas_jugadas = df_users_items.groupby('item_name')['playtime_forever'].sum().reset_index()
df_horas_jugadas.head()

,item_name,playtime_forever
0,神明的一天世界(God's One Day World),6969.0
1,! That Bastard Is Trying To Steal Our Gold !,742.0
2,"""Glow Ball"" - The billiard puzzle game",21.0
3,#SelfieTennis,261.0
4,#SkiJump,0.0


Vamos a extraer los datos de los juegos de steam y eliminemos los que no tengan titulos.

In [55]:
df_steam_games = pd.read_csv('../Data/Data-Extraida/steam_games.csv')
df_steam_games.head()

C:\Users\alexi\AppData\Local\Temp\ipykernel_11992\2495168712.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_steam_games = pd.read_csv('../Data/Data-Extraida/steam_games.csv')


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df_steam_games.dropna(subset=['title'], inplace=True)
df_steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,Strategy,http://steamcommunity.com/app/761140/reviews/?...,Single-player,4.99,False,761140.0,Kotoshiro
88311,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,Action,http://steamcommunity.com/app/761140/reviews/?...,Single-player,4.99,False,761140.0,Kotoshiro
88312,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,Indie,http://steamcommunity.com/app/761140/reviews/?...,Single-player,4.99,False,761140.0,Kotoshiro
88313,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,Casual,http://steamcommunity.com/app/761140/reviews/?...,Single-player,4.99,False,761140.0,Kotoshiro
88314,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,Simulation,http://steamcommunity.com/app/761140/reviews/?...,Single-player,4.99,False,761140.0,Kotoshiro


Vamos a unir los dataframe de horas jugadas con el dataframe de juegos de steam a traves de las columnas **'title** y **'item_name'** para obtener un dataframe de horas jugadas por genero.

In [57]:
df_generos = pd.merge(df_steam_games, df_horas_jugadas, left_on='title',right_on='item_name', how='inner')
df_generos.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,item_name,playtime_forever
0,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,Racing,http://steamcommunity.com/app/282010/reviews/?...,Single-player,9.99,False,282010.0,Stainless Games Ltd,Carmageddon Max Pack,9319.0
1,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,Racing,http://steamcommunity.com/app/282010/reviews/?...,Multi-player,9.99,False,282010.0,Stainless Games Ltd,Carmageddon Max Pack,9319.0
2,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,Racing,http://steamcommunity.com/app/282010/reviews/?...,Steam Trading Cards,9.99,False,282010.0,Stainless Games Ltd,Carmageddon Max Pack,9319.0
3,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,Action,http://steamcommunity.com/app/282010/reviews/?...,Single-player,9.99,False,282010.0,Stainless Games Ltd,Carmageddon Max Pack,9319.0
4,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,http://store.steampowered.com/app/282010/Carma...,1997-06-30,Action,http://steamcommunity.com/app/282010/reviews/?...,Multi-player,9.99,False,282010.0,Stainless Games Ltd,Carmageddon Max Pack,9319.0


In [58]:
df_generos.drop(['publisher','app_name','url','tags','reviews_url','specs','price','early_access','developer','item_name'],axis=1, inplace=True)

In [59]:
df_generos.drop_duplicates(inplace=True)
df_generos.head()

,genres,title,release_date,id,playtime_forever
0,Action,Carmageddon Max Pack,1997-06-30,282010.0,9319.0
24,Indie,Carmageddon Max Pack,1997-06-30,282010.0,9319.0
48,Racing,Carmageddon Max Pack,1997-06-30,282010.0,9319.0
72,Action,Half-Life,1998-11-08,70.0,2682852.0
132,Strategy,Disciples II: Gallean's Return,2006-07-06,1640.0,27397.0


In [60]:
df_generos.reset_index(drop=True,inplace=True)
df_generos.head()

,genres,title,release_date,id,playtime_forever
0,Action,Carmageddon Max Pack,1997-06-30,282010.0,9319.0
1,Indie,Carmageddon Max Pack,1997-06-30,282010.0,9319.0
2,Racing,Carmageddon Max Pack,1997-06-30,282010.0,9319.0
3,Action,Half-Life,1998-11-08,70.0,2682852.0
4,Strategy,Disciples II: Gallean's Return,2006-07-06,1640.0,27397.0


Vamos a covertir a formato de fecha toda la columna **'release_date** que cumplan el formato "YYYY-MM-DD"

In [61]:
df_generos['release_date'] = pd.to_datetime(df_generos['release_date'], format='mixed', errors='coerce')

Agramaos una columna con el año de lanzamiento

In [62]:
df_generos['release_year'] = df_generos['release_date'].dt.year
df_generos.head()

,genres,title,release_date,id,playtime_forever,release_year
0,Action,Carmageddon Max Pack,1997-06-30,282010.0,9319.0,1997.0
1,Indie,Carmageddon Max Pack,1997-06-30,282010.0,9319.0,1997.0
2,Racing,Carmageddon Max Pack,1997-06-30,282010.0,9319.0,1997.0
3,Action,Half-Life,1998-11-08,70.0,2682852.0,1998.0
4,Strategy,Disciples II: Gallean's Return,2006-07-06,1640.0,27397.0,2006.0


In [63]:
df_generos.drop(['title', 'release_date', 'id'],axis=1, inplace=True)
df_generos.head()

,genres,playtime_forever,release_year
0,Action,9319.0,1997.0
1,Indie,9319.0,1997.0
2,Racing,9319.0,1997.0
3,Action,2682852.0,1998.0
4,Strategy,27397.0,2006.0


Alamcenamos el nuevo dataframe en un archivo csv

In [64]:
df_generos.to_parquet('../Data/Data-Endpoints/generos.parquet')